# Understanding the Python code

In [ ]:
# Mount google drive 
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/navar 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/navar'
/content/drive/MyDrive/navar


## Setup

Deep learning is implemented through PyTorch. Running in this in Colab seems to work by default.

In [ ]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.9787, 0.0285, 0.2613],
        [0.6004, 0.1945, 0.4366],
        [0.5072, 0.7711, 0.4547],
        [0.4004, 0.1937, 0.0135],
        [0.1877, 0.6193, 0.3672]])


## NAVAR code

Let's dive straight into the code. It seems that most Python scripts lead back to the main script [NAVAR.py](NAVAR/NAVAR.py) in one way or another.

That file builds two different classes, `NAVAR` and `NAVARLSTM`. We will look at them one by one now.

### `NAVAR`

The `NAVAR` class takes the following inputs:



In [ ]:
import torch.nn as nn

class NAVAR(nn.Module):
    def __init__(self, num_nodes, num_hidden, maxlags, hidden_layers=1, dropout=0):
        """
        Neural Additive Vector AutoRegression (NAVAR) model
        Args:
            num_nodes: int
                The number of time series (N)
            num_hidden: int
                Number of hidden units per layer
            maxlags: int
                Maximum number of time lags considered (K)
            hidden_layers: int
                Number of hidden layers
            dropout:
                Dropout probability of units in hidden layers
        """
        super(NAVAR, self).__init__() # inheret methods of Module class

        self.num_nodes = num_nodes # these are the output nodes 
        self.num_hidden = num_hidden # units per layer
        # MODEL ARCHITECTURE ----
        # This is essentially an MLP composed of multiple hidden layers, each of
        # them a 1D convolution.
        # 1) First layer
        self.first_hidden_layer = nn.Conv1d(num_nodes, num_hidden * num_nodes, kernel_size=maxlags,
                                                  groups=num_nodes)
        # Input tensor: 
        self.dropout = nn.Dropout(p=dropout) # to prevent overfitting
        self.hidden_layer_list = nn.ModuleList() # intialize a list
        self.dropout_list = nn.ModuleList() # intialize a list
        # 2) Append remaining layers
        # Q: Why is kernel_size (filter?) now set to number of units? Unclear
        # Build the list of models:
        for k in range(hidden_layers - 1):
            self.hidden_layer_list.append(
                nn.Conv1d(num_nodes, num_hidden * num_nodes, kernel_size=num_hidden, groups=num_nodes))
            self.dropout_list.append(nn.Dropout(p=dropout))
        # OUTPUT ----
        # Penultimate layer:
        self.contributions = nn.Conv1d(num_nodes, num_nodes * num_nodes, kernel_size=num_hidden, groups=num_nodes)
        self.biases = nn.Parameter(torch.ones(1, num_nodes) * 0.0001) # hard-coded 0.0001?

    def forward(self, x):
        # Initialize first hidden layer ...
        hidden = self.first_hidden_layer(x).clamp(min=0).view([-1, self.num_nodes, self.num_hidden])
        # ... and dropout:
        hidden = self.dropout(hidden)
        # FEED FORWARD ----
        # Then loop through following layers ....
        for i in range(len(self.hidden_layer_list)):
            hidden = self.hidden_layer_list[i](hidden).clamp(min=0).view([-1, self.num_nodes, self.num_hidden])
            hidden = self.dropout_list[i](hidden)
        # ... until reaching the penultimate layer.
        contributions = self.contributions(hidden)
        contributions = contributions.view([-1, self.num_nodes, self.num_nodes, 1])
        # ADDITIVE ---- 
        predictions = torch.sum(contributions, dim=1).squeeze() #+ self.biases 
        contributions = contributions.view([-1, self.num_nodes*self.num_nodes, 1]).squeeze()
        return predictions, contributions